# 201810803 장서령

### 1-1 geth@8446에서 admin.nodeInfo 출력 (네트워크 ID 4이하로 출력되지 않게 하고, 36번으로 출력함)

### 1-1 답
_genesis2.js 파일을 만들어 chainID를  36으로 변경해준 후, _gethNow2.bat 파일을 만들어 port를 8446으로 변경해주었다. 이후 geth network 초기화를 위해  다음의 코드를 실행하였다.
> geth --datadir .\eth init _genesis2.json

그리고 _gethNow2.bat를 띄워놓고 다음을 실행함으로써 chainId와 network가 36으로 변경됨을 확인하였다.

In [179]:
!geth --exec "admin.nodeInfo" attach http://127.0.0.1:8446

{
  enode: "enode://d22b5d9a3a52317e2830a33365e1e9e5bfcbcddffc646eace07d37b72e32ad01e1a876028e18226b5dc821f52874641938c326d0a4cdeeaa44419668b3f925bb@127.0.0.1:38446",
  enr: "0xf89cb840c0eda4080468a9b98324a6a29a4eb49634afee9f56a0a8571e495fd325ef72161885525d314ef39e0a7079f8d0565c19e1aca4749573fb28bf67bc5e5ff24acc3a83636170ccc5836574683ec5836574683f826964827634826970847f00000189736563703235366b31a103d22b5d9a3a52317e2830a33365e1e9e5bfcbcddffc646eace07d37b72e32ad018374637082962e8375647082962e",
  id: "ef4801d4d9b97eb20443610783ac23cdb4254d2d1af0b276a6bca54f5b065d61",
  ip: "127.0.0.1",
  listenAddr: "[::]:38446",
  name: "Geth/jsr/v1.8.20-stable-24d727b6/windows-amd64/go1.11.2",
  ports: {
    discovery: 38446,
    listener: 38446
  },
  protocols: {
    eth: {
      config: {
        chainId: 36,
        eip150Hash: "0x0000000000000000000000000000000000000000000000000000000000000000",
        eip155Block: 0,
        eip158Block: 0,
        homesteadBlock: 0
      },
      difficulty: 1377

### 1-2 geth@8446에서 계정, ether 잔고(잔고가 5 ether 이상 있어야 함),블록번호를 출력

In [180]:
%%writefile src/mid1.2_Balance.js
var ac1 = eth.accounts[4];
var ac2 = eth.accounts[5];
var bal1 = eth.getBalance(ac1);
var bal2 = eth.getBalance(ac2);
console.log('The Accounts is: ',eth.accounts);
console.log('The Coinbase Balance is: ',web3.fromWei(bal1));
console.log('The 2st Balance is: ',web3.fromWei(bal2));
console.log('The Blocknumber: ', eth.blockNumber);

Overwriting src/mid1.2_Balance.js


### 1-2 답

In [181]:
!geth --exec "loadScript('src/mid1.2_Balance.js')" attach http://localhost:8446

The Accounts is:  0xc920e93d60afac15d7e2ca2c3fccf5307b3e227e,0xcf4e8b9f748e55d04fe7fccec6d027f1af0d87cb,0x0a20a345e9fc872fec37c61674c99dd2d10488b1,0xe9605f3b42d8dacef65ab4761e3e12789b312142,0xf734b591f560cc45b186e9fa754001532eedb31a,0x9a01562cca6dfb2f0cf93e77dd6ea33117650032
The Coinbase Balance is:  252.79
The 2st Balance is:  92.21
The Blocknumber:  989
true


### 1-3 & 1-5 geth@8446 coinbase에서 geth@8446 2번째 계정으로 1.11 ether 계좌이체, 계좌이체가 성공했다면 geth@8446의 수신측 계정잔고, 수신측 잔고변화 ether, 블록번호를 출력하세요.

In [182]:
%%writefile src/mid1.3_Tran.js
miner.setEtherbase(eth.accounts[4]);
console.log('coinbase: ', eth.coinbase);
var bal1=eth.getBalance(eth.coinbase);
var bal2=eth.getBalance(eth.accounts[5]);
var ch1=web3.fromWei(bal2,"ether");
console.log('The Sender balance in ether: ', web3.fromWei(bal1,"ether"));
console.log('The Receiver balance in ether: ', ch1);
console.log('Tranaction Count: ', eth.getTransactionCount(eth.coinbase));
var t=eth.sendTransaction({from:eth.coinbase, to:eth.accounts[5], value:1110000000000000000});
console.log('TransactionHash: ',t);
console.log('...mining start');
miner.start(1);admin.sleepBlocks(1);miner.stop();
console.log('mining done...');
var bal1new=eth.getBalance(eth.coinbase);
var bal2new=eth.getBalance(eth.accounts[5]);
var ch2=web3.fromWei(bal2new,"ether");
console.log('- New Sender balance in ether: ', web3.fromWei(bal1new,"ether"));
console.log('- New Receiver balance in ether: ', ch2);
console.log('- The change of Receiver in ether: ', ch2-ch1);
console.log('block number: ', eth.blockNumber);

Overwriting src/mid1.3_Tran.js


### 1-3&1-5 답

In [183]:
!geth --exec "loadScript('src/mid1.3_Tran.js')" attach http://localhost:8446

coinbase:  0xf734b591f560cc45b186e9fa754001532eedb31a
The Sender balance in ether:  252.79
The Receiver balance in ether:  92.21
Tranaction Count:  16
TransactionHash:  0xb79acdd7012e07f9521e1711e7e2bba4e091f2ebbacc6c4daf1562d9e89b2415
...mining start
mining done...
- New Sender balance in ether:  256.68
- New Receiver balance in ether:  93.32
- The change of Receiver in ether:  1.1099999999999994
block number:  990
true


### 1-4 계좌이체의 hash값을 사용해 getTransactionReceipt 출력
### 1-4 답

In [185]:
!geth --exec "eth.getTransactionReceipt('0xb79acdd7012e07f9521e1711e7e2bba4e091f2ebbacc6c4daf1562d9e89b2415')" attach http://localhost:8446

{
  blockHash: "0x6ef66b40f8541b3f887b9959c272e4ad1d270bfbf8c8ba0e7be547a2a4fe0b7a",
  blockNumber: 990,
  contractAddress: null,
  cumulativeGasUsed: 21000,
  from: "0xf734b591f560cc45b186e9fa754001532eedb31a",
  gasUsed: 21000,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  root: "0xe892f2b66b985dacb0822dead9cc343b799cda863319286bc35cb019a8d5c6b7",
  to: "0x9a01562cca6dfb2f0cf93e77dd6ea33117650032",
  transactionHash: "0xb79acdd7012e07f9521e1711e7e2bba4e091f2ebbacc6c4daf1562d9e89b2415",
  transa

### 2-1 컨트랙은 Math.sol로 저장하고, 함수는 powerOf2로 명명해서 프로그램 작성

### 2-1 답

In [186]:
%%writefile src\Math.sol
pragma solidity ^0.4.0;
contract Math{
    function powerOf2 (uint input) public pure returns(uint) {
            uint out =1;
          for (uint i=0; i<input; i++) {
              out *=2;
        }
        return out;
    }
}

Overwriting src\Math.sol


### 2-2 컴파일해서 abi, gas, bin을 출력
### 2-2 답

In [187]:
!solc --abi --bin --gas src\Math.sol


======= src\Math.sol:Math =======
Gas estimation:
construction:
   93 + 44600 = 44693
external:
   powerOf2(uint256):	infinite
Binary: 
608060405234801561001057600080fd5b5060df8061001f6000396000f300608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063bdacc0cf146044575b600080fd5b348015604f57600080fd5b50606c600480360381019080803590602001909291905050506082565b6040518082815260200191505060405180910390f35b600080600060019150600090505b8381101560a95760028202915080806001019150506090565b81925050509190505600a165627a7a72305820b7a23981db3312ef61ba7d69946bc5ef5c68cbc62fe85fa3ff7142444e2fc34e0029
Contract JSON ABI 
[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"powerOf2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}]


### 2-3 Node로 배포 (배포 스크립트 작성)
### 2-3 답(배포)

In [227]:
%%writefile src/MathDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8446"));
}
var _abiArray=[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"powerOf2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}];
var _bin="608060405234801561001057600080fd5b5060df8061001f6000396000f300608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063bdacc0cf146044575b600080fd5b348015604f57600080fd5b50606c600480360381019080803590602001909291905050506082565b6040518082815260200191505060405180910390f35b600080600060019150600090505b8381101560a95760028202915080806001019150506090565b81925050509190505600a165627a7a72305820b7a23981db3312ef61ba7d69946bc5ef5c68cbc62fe85fa3ff7142444e2fc34e0029";
var _contract = new web3.eth.Contract(_abiArray);
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0xf734b591f560cc45b186e9fa754001532eedb31a", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/MathDeploy.js


In [229]:
!node src\MathDeploy.js

0x6cd754b863dEB372E4205dDFCeAfCAF154f1f090


### 2-4 Node로 사용 (사용 스크립트 작성, 8과 32를 입력해서 결과 출력)

In [230]:
%%writefile src/MathUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8446"));
var shelloContract = new web3.eth.Contract([{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"powerOf2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}],
                                      "0x6cd754b863dEB372E4205dDFCeAfCAF154f1f090");
shelloContract.methods.powerOf2(8).call().then(function(str) {console.log(str);});
shelloContract.methods.powerOf2(32).call().then(function(str) {console.log(str);});

Overwriting src/MathUse.js


### 2-4 답

In [231]:
!node src/MathUse.js

256
4294967296


### 2-5 웹페이지 powerOf2.html을 작성 (32를 입력하고, 결과를 화면에 출력하고 캡쳐해서 별도 제출)

In [242]:
%%writefile scripts/powerOf2.html
<!doctype>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/web3@0.20.5/dist/web3.min.js"></script>
<script type="text/javascript">
    //var Web3 = require('web3');  //can be removed
    var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8446"));

    function displaypowerOf2() {
        //web3@0.20.x
        var powerContract = web3.eth.contract([{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"powerOf2","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}]);
        var power = powerContract.at("0x6cd754b863dEB372E4205dDFCeAfCAF154f1f090");
        var power = power.powerOf2(32);
        document.getElementById('result').innerText = "The message from blockchain: " + power;
        document.getElementById('web3coinbase').innerText = 'coinbase: ' + web3.eth.coinbase;
    }
</script>
</head>
<body>
    <h1>Math.powerOf2</h1>
    <button type="button" onClick="displaypowerOf2();">Display PowerOf2</button>
    <div></div>
    <div id="result"></div>
    <div id="web3coinbase"></div>
</body>
</html>

Overwriting scripts/powerOf2.html
